In [6]:
import pandas as pd
import os
import sys
import boto3
import io
import geopandas as gpd
import numpy as np

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_gpkg_from_directory, pull_csv_from_directory, upload_csv_aws
from scripts.utils.write_metadata import append_metadata

In [7]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '2b_reproject/climate_risk/sea_level_rise/exposure/projections/pacific_institute/'

pull_gpkg_from_directory(bucket_name, aws_dir)
pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved GeoPackage as 'climate_pacific_institute_slr_exposure_fire_stations_2000.gpkg' locally
Saved GeoPackage as 'climate_pacific_institute_slr_exposure_fire_stations_2100.gpkg' locally
Saved GeoPackage as 'climate_pacific_institute_slr_exposure_hospitals_2000.gpkg' locally
Saved GeoPackage as 'climate_pacific_institute_slr_exposure_hospitals_2100.gpkg' locally
Saved GeoPackage as 'climate_pacific_institute_slr_exposure_police_stations_2000.gpkg' locally
Saved GeoPackage as 'climate_pacific_institute_slr_exposure_police_stations_2100.gpkg' locally
Saved GeoPackage as 'climate_pacific_institute_slr_exposure_schools_2000.gpkg' locally
Saved GeoPackage as 'climate_pacific_institute_slr_exposure_schools_2100.gpkg' locally
Saved GeoPackage as 'climate_pacific_institute_slr_exposure_superfund.gpkg' locally
Saved GeoPackage as 'climate_pacific_institute_slr_exposure_wastewater_treatment.gpkg' locally
Saved GeoPackage as 'climate_pacific_institute_slr_exposure_wastewater_treatment_2000.gpkg' l

In [8]:
census_vulnerable_slr_tracts = pd.read_csv('climate_pacific_institute_slr_exposure_census_vulnerability.csv')
census_vulnerable_slr_tracts

,CensusBlock,CountyFIPS,BlkArea_m2,Perc_2000,Perc_2100,CensusTract
0,60014017001002,1,7.400145e+04,0.013629,0.103142,6001401700
1,60014017001009,1,1.661440e+04,0.162421,0.818744,6001401700
2,60014017001011,1,2.034830e+04,0.033993,0.145896,6001401700
3,60014017002024,1,2.437019e+04,0.007894,0.054463,6001401700
4,60014017002025,1,1.384990e+04,0.218054,0.323897,6001401700
...,...,...,...,...,...,...
9415,61110073001115,111,6.546174e+04,0.008397,0.037215,6111007300
9416,61110073001120,111,2.331707e+04,0.010518,0.024219,6111007300
9417,61110073001121,111,3.143619e+06,0.000482,0.001053,6111007300
9418,61110073001124,111,2.357066e+04,0.006471,0.271358,6111007300


In [23]:
slr_tracts_columns = census_vulnerable_slr_tracts[['CensusTract', 'Perc_2000', 'Perc_2100']]
slr_tracts_columns = slr_tracts_columns.rename(columns={'CensusTract':'tract','Perc_2000':'percent_2000', 'Perc_2100':'percent_2100'})
slr_tracts_columns

,tract,percent_2000,percent_2100
0,6001401700,0.013629,0.103142
1,6001401700,0.162421,0.818744
2,6001401700,0.033993,0.145896
3,6001401700,0.007894,0.054463
4,6001401700,0.218054,0.323897
...,...,...,...
9415,6111007300,0.008397,0.037215
9416,6111007300,0.010518,0.024219
9417,6111007300,0.000482,0.001053
9418,6111007300,0.006471,0.271358


In [25]:
slr_tracts_grouped = slr_tracts_columns.groupby('tract')[['percent_2000', 'percent_2100']].mean().reset_index()
slr_tracts_grouped

,tract,percent_2000,percent_2100
0,6001401700,0.173551,0.332988
1,6001401900,0.130183,0.389499
2,6001403200,0.010183,0.186697
3,6001403300,0.302783,0.452502
4,6001403400,0.205320,0.411437
...,...,...,...
626,6111004600,0.911470,0.959438
627,6111004702,0.522970,0.805990
628,6111004704,0.235746,0.650934
629,6111005600,0.361981,0.665406


In [19]:
# read in CA census tiger file
ca_tract_county = "s3://ca-climate-index/0_map_data/ca_tracts_county.csv"
ca_tract_county = gpd.read_file(ca_tract_county)
ca_tract_county.columns = ca_tract_county.columns.str.lower()
ca_tract_county = ca_tract_county.applymap(lambda s: s.lower() if type(s) == str else s)
ca_tract_county = ca_tract_county.drop(columns={'field_1', 'geometry', 'countyfp'})

ca_tract_county

C:\Users\jespi\AppData\Local\Temp\ipykernel_25436\1775936636.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ca_tract_county = ca_tract_county.applymap(lambda s: s.lower() if type(s) == str else s)


,tract,county
0,06085504321,santa clara
1,06085504410,santa clara
2,06085507003,santa clara
3,06085507004,santa clara
4,06085502204,santa clara
...,...,...
9124,06059001303,orange
9125,06059001304,orange
9126,06059001401,orange
9127,06013367200,contra costa


In [28]:
ca_tract_county['tract'] = ca_tract_county['tract'].astype(str)
slr_tracts_grouped['tract'] = slr_tracts_grouped['tract'].astype(str)

merged_slr_tracts = pd.merge(ca_tract_county, slr_tracts_grouped, on='tract', how='left')
merged_slr_tracts

,tract,county,percent_2000,percent_2100
0,06085504321,santa clara,NaN,NaN
1,06085504410,santa clara,NaN,NaN
2,06085507003,santa clara,NaN,NaN
3,06085507004,santa clara,NaN,NaN
4,06085502204,santa clara,NaN,NaN
...,...,...,...,...
9124,06059001303,orange,NaN,NaN
9125,06059001304,orange,NaN,NaN
9126,06059001401,orange,NaN,NaN
9127,06013367200,contra costa,NaN,NaN


In [5]:
slr_fire_stations_2000 = gpd.read_file('climate_pacific_institute_slr_exposure_fire_stations_2000.gpkg')
slr_fire_stations_2100 = gpd.read_file('climate_pacific_institute_slr_exposure_fire_stations_2100.gpkg')

slr_fire_stations_2100

,Name,Address,City,Zipcode,Statea,fld2100,fld2000,USCB_STATEFP,USCB_COUNTYFP,USCB_TRACTCE,USCB_GEOID,USCB_NAME,USCB_NAMELSAD,USCB_MTFCC,USCB_FUNCSTAT,USCB_ALAND,USCB_AWATER,USCB_INTPTLAT,USCB_INTPTLON,geometry
0,Foster City Fire Dept,1040 E Hillsdale Blvd,Foster City,94404-1604,CA,1,NaN,06,081,608025,06081608025,6080.25,Census Tract 6080.25,G5020,S,2938144,2134205,+37.5686738,-122.2745605,POINT (-122.27030 37.55976)
1,Burlingame Fire Dept,1399 Rollins Rd,Burlingame,94010-2409,CA,1,NaN,06,081,605100,06081605100,6051,Census Tract 6051,G5020,S,2620759,1639783,+37.5973795,-122.3682086,POINT (-122.36821 37.59177)
2,San Jose Fire Dept,1590 Gold St,Alviso,95002-0000,CA,1,NaN,06,085,504602,06085504602,5046.02,Census Tract 5046.02,G5020,S,58172196,5655714,+37.4495304,-121.9940964,POINT (-121.97612 37.42502)
3,Edgerly Island Fire Dept,1598 Milton Rd,Napa,94559-9771,CA,1,NaN,06,055,201102,06055201102,2011.02,Census Tract 2011.02,G5020,S,149481826,5073187,+38.2693294,-122.3654555,POINT (-122.30817 38.20715)
4,Hunter's Point Fire House,215 Spear St,San Francisco,94105-1612,CA,1,NaN,06,075,061507,06075061507,615.07,Census Tract 615.07,G5020,S,336738,299021,+37.7907573,-122.3891845,POINT (-122.39223 37.79115)
5,Cordelia Fire District,2155 Cordelia Rd,Suisun City,94585-1667,CA,1,NaN,06,095,252402,06095252402,2524.02,Census Tract 2524.02,G5020,S,15039230,252153,+38.2205759,-122.0685278,POINT (-122.05585 38.23447)
6,Long Beach Fire Dept,225 N Marina Dr,Long Beach,90803-4604,CA,1,NaN,06,037,577604,06037577604,5776.04,Census Tract 5776.04,G5020,S,866541,661519,+33.7557499,-118.1100855,POINT (-118.11104 33.74979)
7,Santa Cruz Fire Dept,230 Walnut Ave,Santa Cruz,95060-3817,CA,1,NaN,06,087,101002,06087101002,1010.02,Census Tract 1010.02,G5020,S,1215116,834526,+36.9627695,-122.0189009,POINT (-122.02910 36.97291)
8,Emeryville Fire Marshal,2333 Powell St,Emeryville,94608-1759,CA,1,NaN,06,001,425101,06001425101,4251.01,Census Tract 4251.01,G5020,S,598450,2087491,+37.8382532,-122.3054128,POINT (-122.30080 37.83709)
9,Oakland Fire Training Academy,250 Victory Ct,Oakland,94607-4615,CA,1,NaN,06,001,403302,06001403302,4033.02,Census Tract 4033.02,G5020,S,719239,104126,+37.7937603,-122.2667344,POINT (-122.26570 37.79263)
